In [ ]:
# DON'T USE THIS CELL
import pendulum
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator

local_tz = pendulum.timezone("Asia/Bangkok")
name='Pending'
prefix='Sales_'
path = f'/usr/local/airflow/plugins/{prefix}{name}/'

# datenow_min1 = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")

dag_params = {
    'owner': 'airflow',
    "depends_on_past": False,
    'start_date': datetime(2022, 4, 14, tzinfo=local_tz),
    'email_on_failure': True,
    'email_on_retry': False,
    'email':['duyvq@merapgroup.com', 'vanquangduy10@gmail.com'],
    'do_xcom_push': False,
    'execution_timeout':timedelta(seconds=300)
    # 'retries': 3,
    # 'retry_delay': timedelta(minutes=10),
}

dag = DAG(prefix+name,
          catchup=False,
          default_args=dag_params,
          schedule_interval= '*/30 8-17,22-22 * * *',
          tags=[prefix+name, 'Daily', '30mins']
)

In [1]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [2]:
# execute_bq_query("call view_report.sp_sales()")

In [ ]:
fdom = datetime.now().replace(day=1).strftime("%Y%m%d")
datenow = datetime.now().strftime("%Y%m%d")
datenow_add1 = (datetime.now() + timedelta(1)).strftime("%Y%m%d")

In [ ]:
# Xu ly data XNT
query = f"EXEC pr_OM_RawdataPDAOrderALL_BI '{fdom}', '{datenow}'"
pending = get_ms_df(sql=query)

In [ ]:
try:
    print("data shape", pending.shape)
    assert pending.shape[0] > 0
except AssertionError:
    print("No pending orders")
else:
    print("Do function here")

In [ ]:
# assert pending.shape[0] > 0

766

In [ ]:
execute_bq_query("TRUNCATE TABLE biteam.f_sales_pending")

In [ ]:
bq_values_insert(pending, "f_sales_pending", 2)

1it [00:03,  3.64s/it]


In [ ]:
# Dont Execute this
dummy_start = DummyOperator(task_id="dummy_start", dag=dag)

extract_dms = PythonOperator(task_id="extract_dms", python_callable=extract_dms, dag=dag)

py_etl_to_postgres = PythonOperator(task_id="etl_to_postgres", python_callable=etl_to_postgres, dag=dag)

# hello_task4 = ToCSVMsSqlOperator(task_id='sample-task-4', mssql_conn_id="1_dms_conn_id", sql=sql, database="PhaNam_eSales_PRO", path=path, dag=dag)

dummy_start >> extract_dms >> py_etl_to_postgres